In [1]:
# Author: iTrue
# Date: 2023-04-14
# Description: Scan WiFi SSIDs

import os
import pandas as pd
import subprocess

def get_wifi_networks():
    cmd = 'netsh wlan show networks mode=Bssid'
    output = subprocess.check_output(cmd, shell=True)
    output = output.decode('utf-8')
    networks = []
    network = {}
    for line in output.split('\n'):
        line = line.strip()
        if line.startswith('SSID'):
            if network:
                networks.append(network)
            network = {}
            ssid = line.split(':')[1].strip()
            network['wifiname'] = ssid
        elif line.startswith('BSSID'):
            bssid = line.split(':')[1].strip()
            network['bssid'] = bssid
        elif line.startswith('Authentication'):
            auth = line.split(':')[1].strip()
            network['authentication'] = auth
    if network:
        networks.append(network)
    df = pd.DataFrame(networks)
    return df

def scan_wifi():
    if os.name == 'nt':
        return get_wifi_networks()

    else:
        import network

        wlan = network.WLAN(network.STA_IF)
        wlan.active(True)
        nets = wlan.scan()
        ssids = [n[0].decode() for n in nets]
        return ssids
    
if __name__ == '__main__':
    # import pandas as pd
    # # Create an empty DataFrame
    # df = pd.DataFrame(columns=['wifi name', 'ssid'])
    # ssids = scan_wifi()

    # # Create a list of dictionaries for appending data
    # data = [{'wifi name': ssid[0], 'ssid': ssid[1]} for ssid in ssids]

    # # Concatenate the list of dictionaries to the DataFrame
    # df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)

    # # Print the updated DataFrame
    # print(df)
    print(scan_wifi())


  wifiname authentication bssid
0    Triet  WPA2-Personal    80


In [ ]:
# Author: iTrue
# Date: 2023-04-14
# Description: Connect to WiFi with Python known SSID and password
# !pip install network
import network

def connect_to_wifi(ssid, password):
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    if not wlan.isconnected():
        print('connecting to network...')
        wlan.connect(ssid, password)
        while not wlan.isconnected():
            pass
    print('network config:', wlan.ifconfig())

  Preparing metadata (setup.py) ... done
  Created wheel for network: filename=network-0.1-py3-none-any.whl size=3138 sha256=16657ac64b4813cc65f69b40276d5f6399d65d50308c8999732aa347db88ab5b
  Stored in directory: /Users/trieupham/Library/Caches/pip/wheels/3a/9a/a4/341d3b109494a43a5cdd444ca83be3a4bfe8c1267ad9f85332
Successfully built network


In [7]:
# Author: iTrue
# Date: 2023-04-14
# Scan all devices on the network
# !pip install scapy
# !pip install requests

from scapy.all import ARP, Ether, srp
import requests
# create ARP packet
arp = ARP(pdst="192.168.1.0/24")
# create Ether broadcast packet
ether = Ether(dst="ff:ff:ff:ff:ff:ff")
# combine the two packets
packet = ether/arp

# send the packet and capture response
result = srp(packet, timeout=3, verbose=0)[0]

# parse the response
devices = []
for sent, received in result:
    device = {'ip': received.psrc, 'mac': received.hwsrc}
    try:
        device_name = requests.get(f"https://api.macvendors.com/{received.hwsrc}").text.strip()
    except:
        device_name = 'Unknown'
    device['device_name'] = device_name
    devices.append(device)

# print the devices
print("Available devices in the network:")
print("IP" + " "*18+"MAC"+" "*18+"Device Name")
for device in devices:
    print(f"{device['ip']}\t{device['mac']}\t{device['device_name']}")


Available devices in the network:
IP                  MAC                  Device Name
192.168.1.1	80:69:33:f6:8e:06	HUAWEI TECHNOLOGIES CO.,LTD
192.168.1.22	60:d9:c7:dd:0c:19	Apple, Inc.
192.168.1.34	a2:03:c2:f2:c2:81	{"errors":{"detail":"Not Found"}}
192.168.1.63	9c:b6:d0:ea:77:8d	Rivet Networks
192.168.1.49	c0:b5:d7:0e:c7:63	CHONGQING FUGUI ELECTRONICS CO.,LTD.
192.168.1.68	9a:52:0e:e8:90:2a	{"errors":{"detail":"Not Found"}}
192.168.1.94	9e:f4:0d:33:53:f3	{"errors":{"detail":"Not Found"}}


In [4]:
# Author: iTrue
# Date: 2023-04-14
# Description: Using nmap to scan open ports

!pip install python-nmap
import nmap

# initialize nmap scanner
nm = nmap.PortScanner()

# define the host to scan
target_host = "192.168.1.1"

# perform the scan
nm.scan(target_host, arguments="-sV -p-")

# loop through all the open ports
for port in nm[target_host]["tcp"]:
    if nm[target_host]["tcp"][port]["state"] == "open":
        print(f"Port {port} is open")



Port 53 is open
Port 80 is open
Port 443 is open
Port 27998 is open
Port 37443 is open
Port 37444 is open


In [3]:


from scapy.all import ARP, Ether, srp
import requests

# create ARP packet
arp = ARP(pdst="192.168.1.0/24")
# create Ether broadcast packet
ether = Ether(dst="ff:ff:ff:ff:ff:ff")
# combine the two packets
packet = ether/arp

# send the packet and capture response
result = srp(packet, timeout=3, verbose=0)[0]

# parse the response
devices = []
for sent, received in result:
    device = {'ip': received.psrc, 'mac': received.hwsrc}
    try:
        device_name = requests.get(f"https://api.macvendors.com/{received.hwsrc}").text.strip()
    except:
        device_name = 'Unknown'
    device['device_name'] = device_name
    devices.append(device)

# print the devices in the same format as arp -a command
print("Interface: en0")
for device in devices:
    print(f"{device['ip']}\t{device['mac']}\t{device['device_name']}")


Interface: en0
192.168.1.1	80:69:33:f6:8e:06	HUAWEI TECHNOLOGIES CO.,LTD
192.168.1.63	9c:b6:d0:ea:77:8d	Rivet Networks
192.168.1.68	9a:52:0e:e8:90:2a	{"errors":{"detail":"Not Found"}}


In [5]:
# Author: iTrue
# Date: 2023-04-14
# Description: Create Remote Desktop Connection

# !pip install pyRDP
# Check if pyRDP is installed then install it if not
import importlib
if importlib.util.find_spec('pyRDP') is None:
    !pip install pyRDP

from pyRDP import RDP

# create an instance of the RDP class
rdp = RDP()

your_username = 'BTMAdmin'
your_password = 'Xstore@123'
remote_host_address = '10.2.104.181'

# set the connection parameters
rdp.set_credentials(username='your_username', password='your_password')
rdp.set_destination(address='remote_host_address', port=3389)

# connect to the remote desktop
rdp.connect()

  Obtaining dependency information for pyRDP from https://files.pythonhosted.org/packages/09/8e/f103435f4529450e7c9c98aa76b3439c382f9b11025072d89a8dc59bdcd5/pyrdp-0.1.5-py3-none-any.whl.metadata
  Using cached pyrdp-0.1.5-py3-none-any.whl.metadata (719 bytes)
Using cached pyrdp-0.1.5-py3-none-any.whl (2.8 kB)


ModuleNotFoundError: No module named 'pyRDP'